In [ ]:
from pathlib import Path
from utils.get_input import get_input
import cvxpy as cp


inp = Path("./test_input").read_text()
inp = get_input(13)
inp;

'Button A: X+99, Y+37\nButton B: X+18, Y+26\nPrize: X=9441, Y=5051\n\nButton A: X+32, Y+49\nButton B: X+39, Y+13\nPrize: X=19007, Y=19244\n\nButton A: X+89, Y+60\nButton B: X+13, Y+73\nPrize: X=4445, Y=4731\n\nButton A: X+38, Y+15\nButton B: X+35, Y+92\nPrize: X=5566, Y=7670\n\nButton A: X+24, Y+51\nButton B: X+53, Y+19\nPrize: X=9532, Y=11844\n\nButton A: X+20, Y+57\nButton B: X+47, Y+16\nPrize: X=17980, Y=8523\n\nButton A: X+19, Y+43\nButton B: X+66, Y+41\nPrize: X=12958, Y=15245\n\nButton A: X+16, Y+80\nButton B: X+85, Y+85\nPrize: X=8335, Y=13455\n\nButton A: X+17, Y+97\nButton B: X+28, Y+23\nPrize: X=1710, Y=2235\n\nButton A: X+11, Y+44\nButton B: X+66, Y+41\nPrize: X=7602, Y=2770\n\nButton A: X+92, Y+23\nButton B: X+73, Y+87\nPrize: X=5809, Y=2071\n\nButton A: X+58, Y+15\nButton B: X+15, Y+48\nPrize: X=17943, Y=9671\n\nButton A: X+41, Y+13\nButton B: X+28, Y+59\nPrize: X=9586, Y=5118\n\nButton A: X+80, Y+98\nButton B: X+85, Y+13\nPrize: X=1550, Y=1352\n\nButton A: X+79, Y+99\nBut

In [34]:
%%timeit
import re

machines = inp.split("\n\n")
machines_parsed = []

reg_x_plus = re.compile(r"(?<=(X\+))\d*")
reg_y_plus = re.compile(r"(?<=(Y\+))\d*")

reg_x_equals = re.compile(r"(?<=(X\=))\d*")
reg_y_equals = re.compile(r"(?<=(Y\=))\d*")

for machine in machines:
    machine = machine.split("\n")
    button_a = (int(next(re.finditer(reg_y_plus, machine[0])).group(0)), int(next(re.finditer(reg_x_plus, machine[0])).group(0)))
    button_b = (int(next(re.finditer(reg_y_plus, machine[1])).group(0)), int(next(re.finditer(reg_x_plus, machine[1])).group(0)))
    prize = (int(next(re.finditer(reg_y_equals, machine[2])).group(0)), int(next(re.finditer(reg_x_equals, machine[2])).group(0)))
    machines_parsed.append((button_a, button_b, prize))

1.89 ms ± 12.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
def part1():
    sum = 0
    # This problem is probably convex, right???
    for button_a, button_b, prize in machines_parsed:
        a = cp.Variable(integer=True)
        b = cp.Variable(integer=True)
        con_1 = a * button_a[0] + b * button_b[0] == prize[0]
        con_2 = a * button_a[1] + b * button_b[1] == prize[1]
        prob = cp.Problem(cp.Minimize(a * 3 + b * 1), [a >= 0, b >= 0, con_1, con_2])
        prob.solve()
        if prob.status == "optimal":
            sum += 3 * int(a.value) + int(b.value)
    return sum


part1()


29201

In [23]:
%%timeit
part1()

793 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def part2():
    sum = 0
    # This problem is probably convex, right???
    for button_a, button_b, prize in machines_parsed:
        a = cp.Variable(integer=True)
        b = cp.Variable(integer=True)
        con_11 = a * button_a[0] + b * button_b[0] >= prize[0] + 10000000000000 - 1e-5
        con_12 = a * button_a[0] + b * button_b[0] <= prize[0] + 10000000000000 + 1e-5
        con_21 = a * button_a[1] + b * button_b[1] >= prize[1] + 10000000000000 - 1e-5
        con_22 = a * button_a[1] + b * button_b[1] <= prize[1] + 10000000000000 + 1e-5
        prob = cp.Problem(
            cp.Minimize(a * 3 + b * 1), [a >= 0, b >= 0, con_11, con_22, con_12, con_21]
        )
        prob.solve()
        if prob.status == "optimal":
            sum += 3 * int(a.value) + int(b.value)
    return sum


part2()


104140871044942

In [31]:
%%timeit
part2()

997 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
